In [1]:
import os

from dotenv import load_dotenv
load_dotenv()
import datasets
from datasets import DatasetDict
from datasets import load_dataset
import datasets
import torch
import json
from huggingface_hub import HfApi

DEBUG = False
EPOCHS = 10


from baseline_curricula import upload, split_into_epochs, repeat_for_ten_epochs

In [2]:
import config

In [3]:
from baseline_curricula import get_perplexity_ngram

for dataset_name in config.datasets:
    perplexity_df = get_perplexity_ngram(dataset_name)
    upload("perplexity_increasing.pt", repeat_for_ten_epochs(perplexity_df["perplexity"].sort_values(ascending=True).index.to_list()), dataset_name)


./difficulty_ngram/1/babylm_2024_10m_curriculum
Skipping ./difficulty_ngram/1/babylm_2024_10m_curriculum, already calculated


No files have been modified since last commit. Skipping to prevent empty commit.


In [4]:
from baseline_curricula import get_mattr


for dataset_name in config.datasets:
    mattr_df = get_mattr(dataset_name, n_proc=128)
 #   upload("perplexity_decreasing.pt", split_into_epochs(perplexity_df["perplexity"].sort_values(ascending=False).index.to_list()), dataset)
    upload("mattr_increasing.pt", repeat_for_ten_epochs(mattr_df["mattr"].sort_values(ascending=True).index.to_list()), dataset_name)


./difficulty_mattr/babylm_2024_10m_curriculum
Skipping ./difficulty_mattr/babylm_2024_10m_curriculum, already calculated


No files have been modified since last commit. Skipping to prevent empty commit.


In [5]:
from baseline_curricula import get_mattr
import torch


for dataset_name in config.datasets:
    
    torch.manual_seed(42)
    dataset = load_dataset(dataset_name)
    random_order = [torch.randperm(len(dataset["train"])) for _ in range(0,EPOCHS)]
    
    upload("random.pt", random_order, dataset_name)


No files have been modified since last commit. Skipping to prevent empty commit.


In [6]:
from baseline_curricula import validate_training_duration_limitation

In [7]:


word_count = lambda d: {"words" : len(d["text"].split())}

WORDS_PER_STAGE = 2*10000000 # there are 5 stages: 2*10M words = 100M total seen during training
for dataset_name in config.datasets:
    
    
    df = load_dataset(dataset_name)["train"].map(word_count, num_proc=100).to_pandas()
    curriculum = []
    for stage_name, stage_df in df.groupby("stage"):
        print(stage_name)
        tokens_in_stage = 0
        curriculum_stage = []
        
        i = 0
        while tokens_in_stage < WORDS_PER_STAGE: # the dataset assumes each stage is repeated for multiple epochs. we will therefore run out of documents before reaching the alloted number of tokens. just re-shuffle and continue
                                                 #    we double-check the total number of tokens seen trough the curriculum below
            stage_df_shuffled = stage_df.sample(frac=1, random_state=42+i)
            for idx, row in stage_df_shuffled.iterrows():            
                if (tokens_in_stage + row["words"]) <= WORDS_PER_STAGE:
                    curriculum_stage.append(idx)
                    tokens_in_stage += row["words"]
                else:
                    tokens_in_stage += row["words"] # so that outer loop breaks
                    break
                
            print(i,tokens_in_stage)
            i+=1
        curriculum.extend(torch.chunk(torch.tensor(curriculum_stage), 2)) # so that there are 2 epochs per stage
    validate_training_duration_limitation(dataset_name, curriculum)

    upload("source_difficulty.pt", curriculum, dataset_name)


C1: Child Directed Speech
0 2839591
1 5679182
2 8518773
3 11358364
4 14197955
5 17037546
6 19877137
7 20000002
C2: Unscripted Dialogue
0 1079286
1 2158572
2 3237858
3 4317144
4 5396430
5 6475716
6 7555002
7 8634288
8 9713574
9 10792860
10 11872146
11 12951432
12 14030718
13 15110004
14 16189290
15 17268576
16 18347862
17 19427148
18 20000024
C3: Scripted Dialogue
0 2041868
1 4083736
2 6125604
3 8167472
4 10209340
5 12251208
6 14293076
7 16334944
8 18376812
9 20000002
C4: Wiki
0 1453539
1 2907078
2 4360617
3 5814156
4 7267695
5 8721234
6 10174773
7 11628312
8 13081851
9 14535390
10 15988929
11 17442468
12 18896007
13 20000009
C5: Written English
0 2539489
1 5078978
2 7618467
3 10157956
4 12697445
5 15236934
6 17776423
7 20000002
[loris3/babylm_2024_10m_curriculum -> tensor([   8933,  437545,  471759,  ..., 1168727, 1120429, 1142718])] 99999795 words : 11026808 documents


  0%|          | 0/1 [00:00<?, ?it/s]

source_difficulty.pt:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

In [8]:
curriculum

[tensor([  8933, 437545, 471759,  ...,  33991, 224857, 383641]),
 tensor([156889, 224053, 264396,  ..., 243565,  34977, 399672]),
 tensor([616349, 589845, 627263,  ..., 662138, 606808, 589816]),
 tensor([633353, 671196, 671111,  ..., 589175, 656175, 639012]),
 tensor([764637, 752372, 940387,  ..., 763855, 978969, 895350]),
 tensor([885912, 848925, 790921,  ..., 930966, 792617, 884297]),
 tensor([1076450, 1098670, 1063811,  ..., 1057198, 1056028, 1108266]),
 tensor([1098060, 1102040, 1049606,  ..., 1096482, 1061346, 1094826]),
 tensor([1132501, 1155708, 1169436,  ..., 1135972, 1154654, 1115620]),
 tensor([1127694, 1161190, 1154973,  ..., 1168727, 1120429, 1142718])]

In [9]:
9953773 / 10000000

0.9953773

In [10]:
buzgitdp+ü

NameError: name 'buzgitdp' is not defined

In [ ]:
from baseline_curricula import validate_training_duration_limitation
from itertools import product
import config 

[validate_training_duration_limitation(dataset_name, curriculum_name) for dataset_name, curriculum_name in product(config.datasets, config.curricula)]



Map (num_proc=100):   0%|          | 0/1179014 [00:00<?, ? examples/s]

In [ ]:
#    torch.save(curriculum, "./source_difficulty.pt")
#  api = HfApi()
#         api.upload_file(
#             path_or_fileobj="./source_difficulty.pt",
#             path_in_repo="source_difficulty.pt",
#             repo_id=api.whoami()["name"] + "/" + args["name"],
#             repo_type="dataset"
#         )